   # PREDICTING HOUSE SALE PRICES

***AIM*** - To build an intuition for model based learning, explore how linear regression model work by performing some of the cleaning, transformation and selecting features on data to reduce the error in predicting data.

***Objective*** - To build a model that predict house sale prices accurately by exploring previous data by performing train test split.

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [37]:
%matplotlib inline

In [38]:
data = pd.read_csv("AmesHousing.tsv",delimiter = "\t")

In [39]:
data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Frontage       2440 non-null float64
Lot Area           2930 non-null int64
Street             2930 non-null object
Alley              198 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         29

In [41]:
def transform_features(data):
    return data

In [42]:
def select_features(data):
    return data[["Gr Liv Area","SalePrice"]]

In [43]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
def train_and_test(data):
    train = data[0:1460]
    test = data[1460:]
    numeric_train = train.select_dtypes(include = ["int","float"])
    numeric_test = test.select_dtypes(include = ["int","float"])
    features = numeric_train.columns.drop("SalePrice")
    model = LinearRegression()
    model.fit(train[features],train["SalePrice"])
    predicted_values = model.predict(test[features])
    mse = mean_squared_error(predicted_values,test["SalePrice"])
    RMSE = mse**(1/2)
    return RMSE

In [44]:
transform_df = transform_features(data)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)
rmse

57088.25161263909

Feature Engineering
Handle missing values:
All columns:
Drop any with 5% or more missing values for now.
Text columns:
Drop any with 1 or more missing values for now.
Numerical columns:
For columns with missing values, fill in with the most common value in that column
1: All columns: Drop any with 5% or more missing values for now.

In [45]:
num_missing = data.isnull().sum()

In [46]:
drop_missing_cols = num_missing[num_missing>len(data)/20].sort_values()

In [47]:
data = data.drop(drop_missing_cols.index,axis = 1)

2: Text columns: Drop any with 1 or more missing values for now.

In [48]:
text_mv_counts = data.select_dtypes(include = ["object"]).isnull().sum().sort_values(ascending = False)

In [49]:
drop_missing_cols_2 = text_mv_counts[text_mv_counts >0]

In [50]:
data = data.drop(drop_missing_cols_2.index, axis = 1)

3: Numerical columns: For columns with missing values, fill in with the most common value in that column

In [51]:
## compute column-wise missing value counts
num_missing = data.select_dtypes(include = ["int","float"]).isnull().sum()
fixable_numeric_cols = num_missing[(num_missing < len(data)/20) & (num_missing >0)].sort_values()
fixable_numeric_cols

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

In [52]:
## Compute the most common value for each column in `fixable_nmeric_missing_cols.
replacement_values_dict = data[fixable_numeric_cols.index].mode().to_dict(orient = "records")[0]

In [53]:
replacement_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [54]:
data = data.fillna(replacement_values_dict)

In [55]:
data.isnull().sum().value_counts()

0    64
dtype: int64

 What new features can we create, that better capture the information in some of the features?

In [56]:
years_sold = data["Yr Sold"] - data["Year Built"]
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [57]:
years_since_remod = data["Yr Sold"] - data["Year Remod/Add"]
years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [58]:
# create New columns
data["Years Before Sale"] = years_sold
data["Years Since Remod"] = years_since_remod

## Drop rows with negative values for both of these rows"
data = data.drop([1702,2180,2181], axis = 0)

## No longer need original year columns
data = data.drop(["Year Built","Year Remod/Add"], axis = 1)

Drop columns that aren't useful for ML

In [59]:
## Drop columns that aren't useful for ML
data = data.drop(["PID","Order"], axis = 1)

## drop data that leak info abuut the final sale
data = data.drop(["Mo Sold", "Sale Condition", "Sale Type","Yr Sold"], axis = 1)

Let's update transform_features()

In [60]:
def transform_features(data):
    num_missing = data.isnull().sum()
    drop_missing_cols = num_missing[(num_missing>len(data)/20)].sort_values()
    data = data.drop(drop_missing_cols.index,axis = 1)
    
    text_mv_counts = data.select_dtypes(include = ["object"]).isnull().sum().sort_values(ascending = False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts >0]
    data = data.drop(drop_missing_cols_2.index, axis = 1)
    
    num_missing = data.select_dtypes(include = ["int","float"]).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(data)/20) & (num_missing >0)].sort_values()
    replacement_values_dict = data[fixable_numeric_cols.index].mode().to_dict(orient = "records")[0]
    data = data.fillna(replacement_values_dict)
    
    years_sold = data["Yr Sold"] - data["Year Built"]
    years_since_remod = data["Yr Sold"] - data["Year Remod/Add"]
    
    data["Years Before Sale"] = years_sold
    data["Years Since Remod"] = years_since_remod
    data = data.drop([1702,2180,2181], axis = 0)
    data = data.drop(["Year Built","Year Remod/Add"], axis = 1)
    
    data = data.drop(["PID","Order"], axis = 1)
    data = data.drop(["Mo Sold", "Sale Condition", "Sale Type","Yr Sold"], axis = 1)
    
    return data
def select_features(data):
    return data[["Gr Liv Area", "SalePrice"]]

def train_and_test(data):  
    train = data[:1460]
    test = data[1460:]
    ## You can use `pd.DataFrame.select_dtypes()` to specify column types
    ## and return only those columns as a data frame.
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    ## You can use `pd.Series.drop()` to drop a value.
    features = numeric_train.columns.drop("SalePrice")
    model = LinearRegression()
    model.fit(train[features], train["SalePrice"])
    predictions = model.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

data = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_data = transform_features(data)
filtered_data = select_features(transform_data)
rmse = train_and_test(filtered_data)

rmse
    

    


55275.36731241307

Feature Selection

In [61]:
numerical_data = transform_data.select_dtypes(include = ["int","float"])
numerical_data.head(5)

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years Before Sale,Years Since Remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,212,34,0,0,0,0,0,189900,13,12


In [62]:
abs_corr_coeffs = numerical_data.corr()["SalePrice"].abs().sort_values()
abs_corr_coeffs

BsmtFin SF 2         0.006127
Misc Val             0.019273
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: Sale

In [63]:
## Let's only keep columns with a correlation coefficient of larger than 0.4 (arbitrary, worth experimenting later!)
abs_corr_coeffs[abs_corr_coeffs > 0.4]

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [64]:
## Drop columns with less than 0.4 correlation with SalePrice
transform_data = transform_data.drop(abs_corr_coeffs[abs_corr_coeffs<0.4].index, axis =1)

Which categorical columns should we keep?

In [ ]:
# create a list of column names from documentation that are meant to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"] 

Which colums are currently numerical but need to be encoded as categorical instead(because the numbers don't have aby semantic meaning)

In [ ]:
transform_cat_cols = []
for col in nominal_features:
    if col in transform_data.columns:
        transform_cat_cols.append(col)
        

If a categorical column ha hundreds of unique values(or categories), should we keepit? when we dummy code this column, hundred of columns will need to be added back to the data frame.

In [66]:
## How many unique values in each categorical column?
uniqueness_counts = transform_data[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
## Aribtrary cutoff of 10 unique values (worth experimenting)
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts>10].index
transform_data = transform_data.drop(drop_nonuniq_cols,axis = 1)

In [67]:
## Select just the remaining text columns and convert to categorical
text_cols = transform_data.select_dtypes(include = ["object"])
for col in text_cols:
    transform_data[col] = transform_data[col].astype("category")
## Create dummy columns and add back to the dataframe!
transform_data = pd.concat([transform_data, pd.get_dummies(transform_data.select_dtypes(include = ["category"]))], axis = 1).drop(text_cols,axis = 1)
    

Update Select_features()

In [85]:
def transform_features(data):
    num_missing = data.isnull().sum()
    drop_missing_cols = num_missing[(num_missing>len(data)/20)].sort_values()
    data = data.drop(drop_missing_cols.index,axis = 1)
    
    text_mv_counts = data.select_dtypes(include = ["object"]).isnull().sum().sort_values(ascending = False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts >0]
    data = data.drop(drop_missing_cols_2.index, axis = 1)
    
    num_missing = data.select_dtypes(include = ["int","float"]).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(data)/20) & (num_missing >0)].sort_values()
    replacement_values_dict = data[fixable_numeric_cols.index].mode().to_dict(orient = "records")[0]
    data = data.fillna(replacement_values_dict)
    
    years_sold = data["Yr Sold"] - data["Year Built"]
    years_since_remod = data["Yr Sold"] - data["Year Remod/Add"]
    
    data["Years Before Sale"] = years_sold
    data["Years Since Remod"] = years_since_remod
    data = data.drop([1702,2180,2181], axis = 0)
    data = data.drop(["Year Built","Year Remod/Add"], axis = 1)
    
    data = data.drop(["PID","Order"], axis = 1)
    data = data.drop(["Mo Sold", "Sale Condition", "Sale Type","Yr Sold"], axis = 1)
    
    return data
   


def select_features(data, coeff_threshold=0.4, uniq_threshold=10):
    numerical_data = data.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_data.corr()['SalePrice'].abs().sort_values()
    data = data.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID","MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in transform_data.columns:
            transform_cat_cols.append(col)
        
    uniqueness_counts = data[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    data = data.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = data.select_dtypes(include=['object'])
    for col in text_cols:
        data[col] = data[col].astype('category')
    data = pd.concat([data, pd.get_dummies(data.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return data
def train_and_test(data, k=0):
    numeric_data = data.select_dtypes(include=['integer', 'float'])
    features = numeric_data.columns.drop("SalePrice")
    model = LinearRegression()
    
    if k == 0:
        train = data[:1460]
        test = data[1460:]

        model.fit(train[features], train["SalePrice"])
        predictions = model.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:   
        # Randomize *all* rows (frac=1) from `data` and return
        shuffled_data = data.sample(frac=1, )
        train = data[:1460]
        test = data[1460:]
        
        model.fit(train[features], train["SalePrice"])
        predictions_one = model.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        model.fit(test[features], test["SalePrice"])
        predictions_two = model.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(data):
            train = data.iloc[train_index]
            test = data.iloc[test_index]
            model.fit(train[features], train["SalePrice"])
            predictions = model.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse



data = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_data = transform_features(data)
filtered_data = select_features(transform_data)
rmse = train_and_test(filtered_data,k = 10)

rmse






[30886.57676286703, 25454.031615305434, 27967.992829642353, 24948.98067528672, 26182.026108027825, 25937.15140047241, 25327.5216231397, 47500.99849743592, 23276.50388573612, 26023.676603840744]


28350.54600017542

___Conclusion___ - By selecting proper features and trying out different validations, error on predictions can be reduced therefore improving the accuracy for the model.